In [2]:
# coding: utf-8

# In[2]:


from __future__ import print_function
from functools import reduce
import re
import json

from gensim.models import Word2Vec
from keras.callbacks import ModelCheckpoint
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np

from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import recurrent, LSTM, Dropout, Merge, Dense
from keras.models import Model, Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model 
from keras.models import model_from_json

# In[9]:
import os
import nltk
import string
from math import log
from nltk.corpus import wordnet as wn
from collections import defaultdict, Counter
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from keras.layers.embeddings import Embedding
from keras.layers.core import Dense, RepeatVector
import gc


lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()


docs_corpus = []
with open("/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/documents.json",'r') as docs:
    docs_corpus = json.load(docs)

train_corpus = []
with open("/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/training.json",'r') as training:
    train_corpus = json.load(training)
print("len(train_corpus)",len(train_corpus))

test_for_train = train_corpus[:10000]

dev_corpus = []             
with open("/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/devel.json",'r') as devel:
    dev_corpus = json.load(devel)

test_for_del = dev_corpus[:200]

test_corpus = []             
with open("/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/testing.json",'r') as test:
    test_corpus = json.load(test)
    
test_for_predict = test_corpus

punc = string.punctuation
stopwordsAll = set(stopwords.words('english'))

def qestion_and_answer(question_id):
    question = train_corpus[question_id]['question']
    answer = train_corpus[question_id]['text']
    processed_question = word_tokenize(question)
    processed_answer = word_tokenize(answer)
    para_id = train_corpus[question_id]['answer_paragraph']
    docid = train_corpus[question_id]['docid']
    # print("processed_question",processed_question)
    # print("answer",answer)
    # print("docid",docid)
    return processed_question, processed_answer, para_id,docid


# qestion_and_answer(0)

def doc_to_story(para_id,docid):
    story = []

    doc = docs_corpus[docid]
    para = ""
    for index, para_data in enumerate(doc['text']):
        if index == para_id:
            para = para_data
        sents = sent_tokenize(para)
        # print("sents",sents)
        for sent in sents:
            tokens = word_tokenize(sent)
            # print("tokens",tokens)
            story.append(tokens)
    # print("story",story)
    return story


# doc_to_story(0)

# save [(story, question, answer)]
def prepare_data(train_corpus):
    final_data = []
    print(len(train_corpus))
    for i in range(len(train_corpus)):
        processed_question, answer, para_id,docid = qestion_and_answer(i)
        story = doc_to_story(para_id,docid)
        final_data.append((story, processed_question, answer))
        # print("final_data",final_data)
    return final_data


def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    xs = []
    xqs = []
    ys = []
    for story, query, answer in data:
        story_list = []
        for sent in story:
            for w in sent:
                story_list.append(w)
        x = [word_idx[w] for w in story_list]
        xq = [word_idx[w] for w in query]
        # let's not forget that index 0 is reserved
        y = np.zeros(len(word_idx) + 1)
        for token in answer:
            y[word_idx[token]] = 1
        xs.append(x)
        xqs.append(xq)
        ys.append(y)
    return pad_sequences(xs, maxlen=story_maxlen), pad_sequences(xqs, maxlen=query_maxlen), np.array(ys)


def get_vocab(data,del_data,test_data):
    vocab = set()
    for story, q, answer in data:
        list_words = []
        for sent in story:
            list_words = list_words + sent
        vocab_list = list_words + q + answer
        # print("vocab_list",vocab_list)
        vocab |= set(vocab_list)
    for story, q, answer in del_data:
        list_words = []
        for sent in story:
            list_words = list_words + sent
        vocab_list = list_words + q + answer
        # print("vocab_list",vocab_list)
        vocab |= set(vocab_list)
    for story, q in test_data:
        list_words = []
        for sent in story:
            list_words = list_words + sent
        vocab_list = list_words + q
        # print("vocab_list",vocab_list)
        vocab |= set(vocab_list)
    vocab = sorted(vocab)
    return vocab

data = prepare_data(test_for_train)
print("len(data)", len(data))


def lemmatize(word):
    lemma = lemmatizer.lemmatize(word, 'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word, 'n')
    return lemma


def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    xs = []
    xqs = []
    ys = []
    for story, query, answer in data:
        story_list = []
        for sent in story:
            for w in sent:
                story_list.append(w)
        x = [word_idx[w] for w in story_list]
        xq = [word_idx[w] for w in query]
        # let's not forget that index 0 is reserved
        y = np.zeros(len(word_idx) + 1)
        for w in answer:
            y[word_idx[w]] = 1
        xs.append(x)
        xqs.append(xq)
        ys.append(y)
    return pad_sequences(xs, maxlen=story_maxlen), pad_sequences(xqs, maxlen=query_maxlen), np.array(ys)


def get_paragraph(docid, document_data):
    # get the paragraph that contains the answer
    for item in document_data:
        if item['docid'] == docid:
            document = item['text']
            break
    return document


def term_freqs(document):
    tfs = defaultdict(dict)
    tfs_forward = defaultdict(dict)
    doc_id = 0
    for sentence in document:
        for token in word_tokenize(sentence):
            if token not in stopwordsAll and token not in punc:
                term = lemmatize(token.lower())
                tfs[term][doc_id] = tfs[term].get(doc_id, 0) + 1
                tfs_forward[doc_id][term] = tfs[doc_id].get(term, 0) + 1
        doc_id += 1
    return tfs, doc_id + 1, tfs_forward


def get_okapibm25(tf, total_docment, documents):
    '''Calculate and return term weights based on okapibm25'''
    k1, b, k3 = 1.5, 0.5, 0
    okapibm25 = defaultdict(dict)

    # calculate average doc length
    total = 0
    for d in documents:
        total += len(d)
    avg_doc_length = total / len(documents) * 1.0

    for term, doc_list in tf.items():
        df = len(doc_list)
        for doc_id, freq in doc_list.items():
            # term occurences in query
            # qtf = question.count(term) # SEPCIAL
            qtf = 1.2
            idf = log((total_docment - df + 0.5) / df + 0.5)
            tf_Dt = ((k1 + 1) * tf[term][doc_id]) / (
            k1 * ((1 - b) + b * (len(documents[doc_id]) / avg_doc_length) + tf[term][doc_id]))
            if qtf == 0:
                third = 0
            else:
                third = ((k3 + 1) * qtf) / (k3 + qtf)
                okapibm25[term][doc_id] = idf * tf_Dt * third

    return okapibm25


# find top_k paragraph that may contain the answer
def get_top_k_document(tfidf, query, k, document):
    top_document_id = Counter()
    for token in word_tokenize(query):
        if token not in stopwordsAll:
            term = lemmatizer.lemmatize(token.lower())
            term_tfidf = tfidf[term]
            for docid, weight in term_tfidf.items():
                top_document_id[docid] += weight
    top_document_id = top_document_id.most_common(k)
    top_document = []
    for document_id, weight in top_document_id:
        top_document.append(document_id)
    return top_document

def prepare_test_del(train_corpus):
    final_data = []
    print(len(train_corpus))
    for i in range(len(train_corpus)):
        story = []
        question = train_corpus[i]['question']
        processed_question, answer, para_id, docid = qestion_and_answer(i)
        document = get_paragraph(docid, docs_corpus)
        tfs, total_docment, tfs_forward = term_freqs(document)
        tfidf = get_okapibm25(tfs, total_docment, document)
        top_1 = get_top_k_document(tfidf, question, 1, document)
        for item in top_1:
            # print("item", item)
            story = doc_to_story(item, docid)
            # print("story",story)
        final_data.append((story, processed_question, answer))
        # print("final_data",final_data)
    return final_data

def prepare_test_data(test_corpus):
    final_data = []
    for item in test_corpus:
        question = item['question']
        docid = item['docid']
        processed_question = word_tokenize(question)
        document = get_paragraph(docid,docs_corpus)
        tfs,total_docment,tfs_forward = term_freqs(document)
        tfidf = get_okapibm25(tfs, total_docment,document)
        top_1 = get_top_k_document(tfidf,question,1,document)
        for item in top_1:
            #print("top_1", top_1)
            story = doc_to_story(item, docid)
            #print("story",story)
        final_data.append((story, processed_question))
        #print("final_data",final_data)
    return final_data

del_data = prepare_test_del(test_for_del)
print("len(del_data)", len(del_data))

def vectorize_stories_test(data, word_idx, story_maxlen, query_maxlen):
    xs = []
    xqs = []
    for story, query in data:
        story_list = []
        for sent in story:
            for w in sent:
                story_list.append(w)
        x = [word_idx[w] for w in story_list]
        xq = [word_idx[w] for w in query]
        xs.append(x)
        xqs.append(xq)
    return pad_sequences(xs, maxlen=story_maxlen), pad_sequences(xqs, maxlen=query_maxlen)

print("len(test_for_predict)",len(test_for_predict))
test_data = prepare_test_data(test_for_predict)
print("len(test_data)",len(test_data))



vocab = get_vocab(data,del_data,test_data)
# print("vocab",vocab)
print("len(vocab)", len(vocab))



RNN = recurrent.LSTM
EMBED_HIDDEN_SIZE = 50
WORD2VEC_EMBED_SIZE = 100
SENT_HIDDEN_SIZE = 100
QUERY_HIDDEN_SIZE = 100
BATCH_SIZE = 32
EPOCHS = 40

WORD2VEC_EMBED_SIZE = 100
print('RNN / Embed / Sent / Query = {}, {}, {}, {}'.format(RNN,
                                                           EMBED_HIDDEN_SIZE,
                                                           SENT_HIDDEN_SIZE,
                                                           QUERY_HIDDEN_SIZE))

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
word2idx = dict((c, i + 1) for i, c in enumerate(vocab))
idx2word = dict((i + 1, c) for i, c in enumerate(vocab))
print("len(word2idx)",len(word2idx))
story_maxlen = max(map(len, (x for x, _, _ in data)))
query_maxlen = max(map(len, (x for _, x, _ in data)))

x, xq, y = vectorize_stories(data, word2idx, story_maxlen, query_maxlen)
tx, txq, ty = vectorize_stories(del_data, word2idx, story_maxlen, query_maxlen)

print("Prepare test data...")

test_story, test_question = vectorize_stories_test(test_data, word2idx, story_maxlen, query_maxlen)

test_question_data = [test_story, test_question]

del data
del del_data
gc.collect()

#word2vec
modelname = "/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/mywvmodel.model"
model = Word2Vec.load(modelname)  # 3个文件放在一起：Word60.model   Word60.model.syn0.npy   Word60.model.syn1neg.npy
print("read model successful")
embedding_weights = np.zeros((vocab_size, WORD2VEC_EMBED_SIZE))
for word, index in word2idx.items():
    try:
        embedding_weights[index, :] = model[word.lower()]
    except KeyError:
        pass  # keep as zero (not ideal, but what else can we do?)

del model
del word2idx
gc.collect()

#former

# print('vocab = {}'.format(vocab))
print('x.shape = {}'.format(x.shape))
print('xq.shape = {}'.format(xq.shape))
print('y.shape = {}'.format(y.shape))
print('story_maxlen, query_maxlen = {}, {}'.format(story_maxlen, query_maxlen))




len(train_corpus) 43379
10000
len(data) 10000
200
len(del_data) 200
len(test_for_predict) 3618
len(test_data) 3618
len(vocab) 50089
RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100
len(word2idx) 50089
Prepare test data...
read model successful


/Users/shimei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:369: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


x.shape = (10000, 1305)
xq.shape = (10000, 60)
y.shape = (10000, 50090)
story_maxlen, query_maxlen = 1305, 60


In [ ]:
'''
print('Build model...')

qenc = Sequential()
qenc.add(Embedding(output_dim=WORD2VEC_EMBED_SIZE, input_dim=vocab_size,
                   weights=[embedding_weights], mask_zero=True))
qenc.add(LSTM(EMBED_HIDDEN_SIZE, input_shape=(None,story_maxlen), return_sequences=False))
qenc.add(Dropout(0.3))

aenc = Sequential()
aenc.add(Embedding(output_dim=WORD2VEC_EMBED_SIZE, input_dim=vocab_size,
                   weights=[embedding_weights], mask_zero=True))
aenc.add(LSTM(EMBED_HIDDEN_SIZE, input_shape=(None,query_maxlen), return_sequences=False))
#aenc.add(RepeatVector(story_maxlen))
aenc.add(Dropout(0.3))

model = Sequential()
model.add(Merge([qenc, aenc], mode="sum"))
model.add(Dense(vocab_size, activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy",
              metrics=["accuracy"])



print('Training')

model.fit([x, xq], y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.05)
loss, acc = model.evaluate([tx, txq], ty,
                           batch_size=BATCH_SIZE)
print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))

print("Saving model...")
# serialize model to JSON
model_json = model.to_json()
with open("predictor_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("predictor_model.h5")
print("Saved model to disk")
 
# later...
'''

In [3]:
# load json and create model
json_file = open('predictor_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("predictor_model.h5")
print("Loaded model from disk")

print('Testing...')
outputs = loaded_model.predict(test_question_data, batch_size = 1)
print("type(outputs)",type(outputs))




/Users/shimei/anaconda3/lib/python3.6/site-packages/keras/engine/topology.py:1271: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


Loaded model from disk
Testing...
type(outputs) <class 'numpy.ndarray'>


In [7]:
import spacy
from __future__ import unicode_literals
from bs4 import BeautifulSoup
nlp = spacy.load('en_core_web_sm')

def get_top_k_document_text(tfidf, query, k, document):
    top_document_id = Counter()
    for token in word_tokenize(query):
        if token not in stopwordsAll:
            term = lemmatizer.lemmatize(token.lower())
            term_tfidf = tfidf[term]
            for docid, weight in term_tfidf.items():
                top_document_id[docid] += weight
    top_document_id = top_document_id.most_common(k)
    top_document = []
    for document_id, weight in top_document_id:
        top_document.append(document[document_id])
    if top_document == []:
        print("top_document is null.")
        top_document.append(document[2])
    return top_document

def prepare_process_data(test_corpus, index):
    final_data = []
    question = test_corpus[index]['question']
    docid = test_corpus[index]['docid']
    processed_question = word_tokenize(question)
    document = get_paragraph(docid,docs_corpus)
    tfs,total_docment,tfs_forward = term_freqs(document)
    tfidf = get_okapibm25(tfs, total_docment,document)
    top_1 = get_top_k_document_text(tfidf,question,1,document)
    final_data.append(top_1)
    #print("final_data",final_data)
    return final_data

count = 0
predictions = []
for index,answer_id in enumerate(outputs):
    #print(index, answer_id)
    story = prepare_process_data(test_for_predict,index)
    #print("story",story)
    doc = ""
    sent1 = []
    for sents in story:
        for sent in sents:
            #print("sent",sent)
            doc = nlp(sent) 
            sent1 = sent
    id_list = []
    for item in answer_id:
        id_list.append(item)
    index1 = np.argmax(id_list)
        #print("type(number)",type(number))
        #print("number",number)
    text1 = idx2word[index1] 
    while text1.lower() not in sent1.lower() or text1.lower() in stopwordsAll or text1 in punc or text1 == r"``" or text1 == "'s" or text1 == "''" or text1 == r"_______" or text1 == r"____" or text1 == r"___" or text1 == r"'～" or text1 == r"“" or text1 == r"..":
            del id_list[index1]
            #new_answer_id = id_list[:index1] + id_list[index1+1:]
            index1 = np.argmax(id_list)
            text1 = idx2word[index1]
    for entity in doc.ents:
        #print("entity",entity)
        #print("text1.lower()",text1.lower(),"entity.text.lower()",entity.text.lower())
        if text1.lower() in entity.text.lower():
            print("Bingo")
            text1 = entity.text.lower()
    for noun_chunk in doc.noun_chunks:
        #print("noun_chunk",noun_chunk)
        #print("text1.lower()",text1.lower(),"noun_chunk.text.lower()",noun_chunk.text.lower())
        if text1.lower() in noun_chunk.text.lower():
            print("Bingo")
            text1 = noun_chunk.text.lower()
    print("index",index,"text1",text1)
    answer_str = (index,text1)
    predictions.append(answer_str)
    count += 1
print("count",count)




Bingo
index 0 text1 early web browsers
Bingo
Bingo
index 1 text1 the world wide web foundation
Bingo
index 2 text1 an unsupported type
Bingo
index 3 text1 some web developers
Bingo
index 4 text1 the user
Bingo
index 5 text1 the industry's first browser war
Bingo
index 6 text1 hyperlinks
Bingo
index 7 text1 quick and easy ways
Bingo
index 8 text1 browser software
Bingo
index 9 text1 the entire browser market
Bingo
index 10 text1 january 2009
Bingo
index 11 text1 may 2012
Bingo
index 12 text1 available web browsers
Bingo
index 13 text1 the sales
Bingo
index 14 text1 some web developers
Bingo
index 15 text1 its first beta release
Bingo
index 16 text1 january 2009
Bingo
index 17 text1 this process
Bingo
index 18 text1 their engine default
Bingo
index 19 text1 local files
Bingo
index 20 text1 minimal, text-based user interfaces
Bingo
index 21 text1 computer manufacturers
Bingo
index 22 text1 august 2011
index 23 text1 ou
Bingo
index 24 text1 first
Bingo
Bingo
index 25 text1 live bookmarks
i

Bingo
index 212 text1 more than 100 police officers
Bingo
index 213 text1 water bottles
Bingo
index 214 text1 protests
index 215 text1 held
Bingo
Bingo
index 216 text1 about 10 members
index 217 text1 ear
Bingo
index 218 text1 its staff
Bingo
index 219 text1 2005
index 220 text1 willing
Bingo
index 221 text1 the torchbearers
Bingo
Bingo
index 222 text1 the 1989 tiananmen square protests
Bingo
index 223 text1 whose brother
Bingo
index 224 text1 their struggle
Bingo
Bingo
index 225 text1 several onlookers
Bingo
Bingo
index 226 text1 american and chinese olympic officials
Bingo
Bingo
index 227 text1 san francisco
Bingo
index 228 text1 breached security
index 229 text1 expelled
Bingo
Bingo
index 230 text1 nazism's swastika
Bingo
index 231 text1 several protests
index 232 text1 officials
Bingo
index 233 text1 chinese diplomats
Bingo
index 234 text1 a ceremony
Bingo
index 235 text1 1989
index 236 text1 a.
Bingo
index 237 text1 the world
Bingo
Bingo
index 238 text1 official olympic insignia
B

Bingo
index 421 text1 vibration
Bingo
index 422 text1 year-end 2016
Bingo
index 423 text1 dates
Bingo
Bingo
index 424 text1 the civic centre
Bingo
Bingo
index 425 text1 the years
Bingo
Bingo
index 426 text1 the 1930s
Bingo
Bingo
index 427 text1 french
index 428 text1 fro
Bingo
index 429 text1 the record
Bingo
Bingo
index 430 text1 the medieval merchant's house
Bingo
Bingo
index 431 text1 southampton water
Bingo
Bingo
Bingo
index 432 text1 the port of southampton
Bingo
index 433 text1 1620
index 434 text1 han
Bingo
index 435 text1 archaeological excavations
Bingo
Bingo
index 436 text1 the 13th century
Bingo
index 437 text1 carbon emissions
Bingo
index 438 text1 the west quay region
Bingo
Bingo
index 439 text1 southampton itchen
index 440 text1 yi
Bingo
index 441 text1 the metropolitan area
Bingo
Bingo
Bingo
Bingo
index 442 text1 southampton airport (parkway) railway station
index 443 text1 fro
Bingo
Bingo
index 444 text1 2004 figures
Bingo
Bingo
index 445 text1 the mayflower theatre
ind

index 639 text1 ring
Bingo
index 640 text1 a fortified settlement
Bingo
index 641 text1 the record
Bingo
index 642 text1 shirley
Bingo
Bingo
index 643 text1 tudor house museum
Bingo
index 644 text1 1066
Bingo
Bingo
Bingo
index 645 text1 the cobden bridge
index 646 text1 pea
Bingo
index 647 text1 the original bridge
Bingo
index 648 text1 european destinations
Bingo
Bingo
index 649 text1 the council leisure executives
index 650 text1 visiting
index 651 text1 k.
Bingo
Bingo
index 652 text1 the 2001 census
index 653 text1 xi
Bingo
index 654 text1 a further part
Bingo
Bingo
index 655 text1 16.2 percent
index 656 text1 ex
Bingo
index 657 text1 totton to southampton central station
Bingo
index 658 text1 several smaller music venues
Bingo
index 659 text1 1879 to 1949
index 660 text1 became
Bingo
index 661 text1 october 2014
Bingo
index 662 text1 this third stage
index 663 text1 han
Bingo
Bingo
index 664 text1 hawthorns urban wildlife centre
Bingo
Bingo
index 665 text1 stagecoach
Bingo
Bingo
in

Bingo
index 849 text1 over 200 known objects
Bingo
index 850 text1 the magnetic fields
index 851 text1 r.
index 852 text1 farther
index 853 text1 r.
Bingo
index 854 text1 the spacecraft's closest approach
Bingo
index 855 text1 ocean
Bingo
index 856 text1 the spacecraft's closest approach
index 857 text1 lead
Bingo
index 858 text1 traces
Bingo
Bingo
index 859 text1 john couch adams
Bingo
index 860 text1 the general pattern
Bingo
Bingo
index 861 text1 the second-most-intense storm
Bingo
index 862 text1 its higher internal heating
index 863 text1 event
Bingo
index 864 text1 unexpected changes
Bingo
Bingo
index 865 text1 saturn
Bingo
index 866 text1 convincing astronomers
Bingo
index 867 text1 this suggestion
Bingo
index 868 text1 subsequent observations
Bingo
index 869 text1 1989
Bingo
index 870 text1 2007
Bingo
index 871 text1 varying compositions
Bingo
index 872 text1 the solar system
Bingo
Bingo
index 873 text1 about 12,500 years
Bingo
index 874 text1 the persistence
Bingo
index 875 te

...

Bingo
Bingo
index 2960 text1 jane anderson
Bingo
Bingo
index 2961 text1 windows 95
Bingo
index 2962 text1 writable cd technology
index 2963 text1 ci
index 2964 text1 respectively
Bingo
Bingo
index 2965 text1 digital research
Bingo
index 2966 text1 2009
index 2967 text1 help
Bingo
index 2968 text1 terry oyama's original design
index 2969 text1 sol
Bingo
index 2970 text1 many non-apple software developers
Bingo
index 2971 text1 these materials
Bingo
index 2972 text1 its innovative design
Bingo
Bingo
Bingo
Bingo
Bingo
index 2973 text1 windows xp
Bingo
index 2974 text1 the original 128k mac
index 2975 text1 given
Bingo
index 2976 text1 july 1, 2013
Bingo
Bingo
index 2977 text1 a lisa
Bingo
index 2978 text1 powerbooks
Bingo
Bingo
index 2979 text1 more than 140,000 pre-orders
Bingo
index 2980 text1 the macintosh office
Bingo
index 2981 text1 the macintosh plus
Bingo
index 2982 text1 fewer slots
Bingo
Bingo
index 2983 text1 jobs
Bingo
Bingo
Bingo
index 2984 text1 crt displays
Bingo
Bingo
inde

Bingo
index 3177 text1 his own life
Bingo
index 3178 text1 archaic roman society
Bingo
Bingo
index 3179 text1 who
Bingo
index 3180 text1 all official business
Bingo
Bingo
Bingo
index 3181 text1 magic spells
Bingo
index 3182 text1 the elite classes
index 3183 text1 official
Bingo
index 3184 text1 the early stages
Bingo
Bingo
index 3185 text1 latin literature
Bingo
Bingo
index 3186 text1 fortuna primigenia
Bingo
index 3187 text1 no evidence
Bingo
index 3188 text1 imported mystery religions
Bingo
Bingo
Bingo
Bingo
index 3189 text1 a vestal's dress
Bingo
index 3190 text1 bailiffs
Bingo
index 3191 text1 the architecture
Bingo
Bingo
index 3192 text1 unofficial bacchanalia cults
index 3193 text1 offered
index 3194 text1 throughout
index 3195 text1 app
Bingo
index 3196 text1 its lawful offices
Bingo
index 3197 text1 her privileges
Bingo
index 3198 text1 the senate
Bingo
index 3199 text1 the republic's collapse
Bingo
index 3200 text1 religious terms
index 3201 text1 serve
index 3202 text1 din
B

Bingo
Bingo
index 3386 text1 decades
index 3387 text1 like
Bingo
index 3388 text1 creatures
Bingo
index 3389 text1 detailed statutes
Bingo
index 3390 text1 appellate decisions
index 3391 text1 able
Bingo
index 3392 text1 every possible situation
index 3393 text1 und
index 3394 text1 automatic
Bingo
index 3395 text1 a politically motivated "absurdity
Bingo
index 3396 text1 grammar
Bingo
index 3397 text1 the tuscan-based language
Bingo
index 3398 text1 linguistic authority
index 3399 text1 peak
index 3400 text1 eve
Bingo
index 3401 text1 more rapid and more pervasive phonological change
index 3402 text1 hi
index 3403 text1 often
Bingo
index 3404 text1 any standard varieties
index 3405 text1 developing
Bingo
index 3406 text1 1996
Bingo
index 3407 text1 institutional support
Bingo
Bingo
index 3408 text1 northern germany
Bingo
index 3409 text1 25.1, 1945
index 3410 text1 per
Bingo
index 3411 text1 each country's specific variety
Bingo
index 3412 text1 modern standard italian
Bingo
index 341

Bingo
index 3602 text1 roughly 620–740 nanometres
Bingo
index 3603 text1 good fortune
Bingo
index 3604 text1 july 17, 1791
Bingo
Bingo
index 3605 text1 vincent van gogh
Bingo
index 3606 text1 at least 1510
Bingo
index 3607 text1 cactus plants
Bingo
Bingo
index 3608 text1 the forbidden city
Bingo
Bingo
index 3609 text1 chinese new year celebrations
Bingo
Bingo
index 3610 text1 five other colors
Bingo
Bingo
index 3611 text1 the 4th century
Bingo
Bingo
index 3612 text1 the atlanta braves
index 3613 text1 hose
index 3614 text1 iz
Bingo
index 3615 text1 the sayings
Bingo
index 3616 text1 chemists
Bingo
index 3617 text1 the 18th and early 19th century
count 3618
predictions[0] (0, 'early web browsers')
len(predictions) 3618


NameError: name 'csv' is not defined

In [8]:
import csv
headers = ['id', 'answer']
print("predictions[0]",predictions[0])
print("len(predictions)",len(predictions))
rows = predictions
with open('output_v1.csv','w') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(rows)
    
print("Finishing writing into output.txt!")



predictions[0] (0, 'early web browsers')
len(predictions) 3618
Finishing writing into output.txt!
